# Acquisizione Dati Partite

In [11]:
import sys
import os
import time
import logging
import importlib
from datetime import datetime
from selenium import webdriver

import sys
print(sys.executable)

import importlib.util
print(importlib.util.find_spec("undetected_chromedriver"))

# Aggiungiamo i moduli locali
if 'modules' not in sys.path:
    sys.path.append(os.path.abspath('modules'))

try:
    from modules import get_matches_per_day
    from modules import db_module
    from modules import fetching
    
    # Ricarica moduli per attivare le nuove barre
    importlib.reload(get_matches_per_day)
    importlib.reload(db_module)
    importlib.reload(fetching)
    
    from modules.fetching import process_date
    
    print("Moduli ricaricati. Sistema di checkpoint attivo.")
except ImportError as e:
    print(f"Errore nell'importazione dei moduli: {e}")

# Silenziamo i log informativi
logging.basicConfig(
    level=logging.WARNING, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)

c:\Users\nicol\Desktop\Progetti\pySofa\.venv\Scripts\python.exe
ModuleSpec(name='undetected_chromedriver', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000014813DC7680>, origin='c:\\Users\\nicol\\Desktop\\Progetti\\pySofa\\.venv\\Lib\\site-packages\\undetected_chromedriver\\__init__.py', submodule_search_locations=['c:\\Users\\nicol\\Desktop\\Progetti\\pySofa\\.venv\\Lib\\site-packages\\undetected_chromedriver'])
Moduli ricaricati. Sistema di checkpoint attivo.


### Configurazione Parametri
Imposta la data di interesse e le opzioni del browser.

In [12]:
# RANGE DI DATE (Formato YYYY-MM-DD)
START_DATE = '2025-01-09'
END_DATE = '2026-02-09'

# Opzioni Selenium
HEADLESS_MODE = True

# Nome del file di checkpoint (verrà creato nella cartella dello script)
CHECKPOINT_FILE = "last_processed_date.txt"

### Funzioni di Utility
Setup del driver e gestione del workflow.

### Esecuzione
Avvia il processo di download.

In [13]:
from datetime import datetime, timedelta
from tqdm.auto import tqdm
import os

# 1. Calcolo date originali
start_dt = datetime.strptime(START_DATE, '%Y-%m-%d')
end_dt = datetime.strptime(END_DATE, '%Y-%m-%d')

# 2. Controllo Checkpoint: se esiste, sovrascrive la data di inizio
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, "r") as f:
        saved_date_str = f.read().strip()
        try:
            saved_dt = datetime.strptime(saved_date_str, '%Y-%m-%d')
            # Inizia dal giorno DOPO l'ultimo salvato
            new_start = saved_dt + timedelta(days=1)
            # Solo se il checkpoint è dentro il range attuale lo usiamo
            if start_dt < new_start <= end_dt:
                start_dt = new_start
                print(f"--- Ripresa dal checkpoint: Inizio da {start_dt.strftime('%Y-%m-%d')} ---")
            elif new_start > end_dt:
                print("--- Tutte le date nel range risultano già elaborate nel checkpoint. ---")
        except:
            print("Checkpoint corrotto, inizio dalla data definita nei parametri.")

# 3. Preparazione lista date effettive da elaborare
date_list = []
current = start_dt
while current <= end_dt:
    date_list.append(current.strftime('%Y-%m-%d'))
    current += timedelta(days=1)

if not date_list:
    print("Nessuna nuova data da elaborare.")
else:
    # Formato barra: tempo rimasto e data a destra
    c_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [Mancano: {remaining}] {postfix}'
    pbar_total = tqdm(date_list, desc="Progresso", position=0, leave=True, bar_format=c_format)

    for curr_str in pbar_total:
        pbar_total.set_postfix_str(f"Data: {curr_str}")
        
        # ELABORAZIONE
        success = process_date(curr_str, headless_mode=HEADLESS_MODE)
        
        # 4. SALVATAGGIO CHECKPOINT: solo dopo che il giorno è finito con successo
        if success:
            with open(CHECKPOINT_FILE, "w") as f:
                f.write(curr_str)
        else:
            print(f"--- ATTENZIONE: Elaborazione fallita per {curr_str}. Checkpoint NON aggiornato. ---")
            break # Fermiamo l'esecuzione per evitare di saltare giorni

--- Ripresa dal checkpoint: Inizio da 2025-03-10 ---


Progresso:   0%|          | 0/337 [Mancano: ?] , Data: 2025-03-10

2026-02-11 14:49:11,286 - WARNING - JSON ricevuto per 2025-03-10 ma senza 'events'. Chiavi: ['error']
2026-02-11 14:49:11,332 - WARNING - [2025-03-10] JSON ricevuto ma senza 'events'. Contenuto errore: {'code': 403, 'reason': 'challenge'}


Progresso:   0%|          | 0/337 [Mancano: ?] , Data: 2025-03-10

--- ATTENZIONE: Elaborazione fallita per 2025-03-10. Checkpoint NON aggiornato. ---


### Stampa primi 10 record per tabella

In [14]:
import db_module
import pandas as pd

conn = db_module.create_connection()
if conn:
    try:
        # Query per gli incidenti (ultimi 10)
        print("--- Ultimi 10 Incidenti (Tabella Colonne) ---")
        query = "SELECT match_id, time, incident_type, player_name, home_score, away_score FROM match_incidents_column LIMIT 10"
        df_inc = pd.read_sql(query, conn)
        display(df_inc)
    except Exception as e:
        print(f"Errore SQL: {e}")
    finally:
        conn.close()


ImportError: attempted relative import with no known parent package